# Chap06 기계학습 응용효과 검증
- YOLOv4 저자의 github 코드를 받아서 대량 증강 이미지를 이용해 학습을 진행합니다
- 코드를 조금 수정해서 validation 데이터셋을 연속, 추론하고 라벨링 도구를 이용해 시각화하여 결과를 검증해봅니다
- 마지막 주차에는 python 코딩 실습이 거의 없습니다
  - 다른 환경의 딥러닝 도구를 사용하지만 예시일 뿐입니다
  - 5주차까지 만든 대량 생산 데이터세트의 효과를 검증하기 위한 도구로서 사용합니다
  - 따라서 다른 여러분들께서 주로 사용하시는 딥러닝 프레임워크가 있다면 그곳에 맞게 사용하셔도 됩니다

## nvidia-smi
- nvidia system monitoring interface
- python 코드가 아니라 별도의 실행파일 입니다
- 따라서 ! 를 붙여서 실행합니다
- K80(keppler 아키텍쳐) 인지 T4(Tesla 아키텍쳐) 인지 확인합니다
  - 별도 colab 구독 서비스에 가입하지 않았다면 이 둘중 하나로 배정됩니다
  - darknet을 빌드할 때 이 정보를 알아야 합니다

In [ ]:
! nvidia-smi

## C++ 에 대해 잠깐만 짚고 넘어가겠습니다
- 런타임에 기계어로 번역되는 python과 달리 런타임 이전에 번역되는 빌드타임이 별도로 
존재하고 이 때 별도의 실행파일을 만듭니다
  - python은 아래와 같이 python 이라는 실행파일을 이용해 test.py 라는 소스를 바로 실행합니다
  ```
  python test.py
  ```
  - C++ 은 빌드타임에 생성된 실행파일을 직접 실행합니다
  ```
  ./test
  ```
- 실행파일을 만드는 설정파일이 Makefile 입니다
  - 아주간단한 예를들어 실행파일의 이름을 설정할 수 있습니다
- colab에 할당된 GPU 기종에 따라 이 Makefile 설정이 필요합니다
  - `이 부분 때문에 짚고 넘어갔습니다`

## darknet 빌드
- darknet이란?
- YOLO 원저자가 만든 오픈소스 딥러닝 프레임워크
- 원저자는 YOLOv3 이후로 연구를 중단하고 이후 YOLOv4 의 저자가 오리지널 darknet을 fork 하여 만든 darknet이 더 널리 쓰이고 있음 
- https://github.com/AlexeyAB/darknet
  - YOLO License 2.0
  - https://github.com/AlexeyAB/darknet/blob/yolov4/LICENSE
- YOLOv4 의 공저자가 이후 YOLOv7 을 발표하였으나 darknet 에서 구현하지는 않고 ultralytics 의 Pytorch 프로젝트로부터 fork 하여 구현
  - GPL 3.0 

In [ ]:
! git clone https://github.com/AlexeyAB/darknet.git
! cd darknet && git reset --hard HEAD
! cd darknet && git checkout tags/yolov4
! cd darknet && wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

# Makefile 수정
- ARCH 주석이 여러개 있는데 NVIDIA T4로 할당 되었다면 아래 하나만 풀어줍니다

```
GPU=1
CUDNN=1
OPENCV=1

ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

```

In [ ]:
! cd darknet && make

# 이미지 1장 추론해보기

In [ ]:
! wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
! tar xvf VOCtrainval_11-May-2012.tar

In [ ]:
# labels 디렉토리를 만들어야 추론 결과가 이곳에 기록되는 구조입니다
! mkdir /content/VOCdevkit/VOC2012/labels
! cd darknet && ./darknet detector test ./cfg/coco.data ./cfg/yolov4-tiny.cfg ./yolov4-tiny.weights /content/VOCdevkit/VOC2012/JPEGImages -dont_show  -save_labels

# 뭔가 의도한대로 돌아가지 않습니다
- 원본 코드는 이미지 1장만 추론할 수 있도록 구현되어 있기 때문입니다
- 아래와 같이 src/detector.c 의 test_detector() 를 수정해봅니다
- yolov4 태그로 checkout한 src/detector.c 기준 line 1645 ~ 1748을 아래 코드로 치환
  - 치환 완료하고 나서 line 386 에 아래 코드 수정(net.max_batches < 1000000)
  ```cpp
          if ((iteration >= (iter_save + 10000) || iteration % 10000 == 0) ||
            (iteration >= (iter_save + 1000) || iteration % 1000 == 0) && net.max_batches < 1000000)
  ```
  - 치환 완료하고 나서 파일의 최상단에 아래 include도 추가
  ```cpp
  #include <dirent.h>
  #include <string.h>
  ```
- stdin 으로 파일 경로입력 받는 것을 지정된 디렉토리를 순회하면서 연속적으로 이미지파일을 추론하는 것으로 변경한 코드입니다

```cpp
	
	DIR *dir;
	struct dirent *ent;

	// 실행 파라미터로 받은 filename 을 디렉토리이름으로 대신 사용합니다
	// 변수명은 filename이지만 사실은 디렉토리이름 입니다
	if ((dir = opendir (filename)) != NULL) {
		/* print all the files and directories within directory */
		// 디렉토리의 모든 파일에 대해 루프를 돌면서 처리합니다
		while ((ent = readdir (dir)) != NULL) {

			// jpg 로 끝나는 파일이 아니면 무시합니다. strcspn 함수는 찾는 문자열이 나오지 않으면 원본 문자열의 길이를 리턴합니다
			int index = strcspn (ent->d_name, "jpg");
			if( index == strlen(ent->d_name)){
					continue;
			}

			// 기존에 input이라고 되어있는 변수 대신 file_path 라는 변수를 사용해 이미지를 한장씩 읽습니다
			// filename이 사실은 디렉토리 경로이기 때문에 사실 아래 코드는 python의 os.path.join(filename, ent->d_name) 과 같습니다
			// ent->d_name은 디렉토리 내에 있는 파일 이름이기 때문에 os.path.join(디렉토리,파일명) 과 같습니다
			char file_path[256] = {0,};
			strcat(file_path, filename);
			strcat(file_path, "/");
			strcat(file_path, ent->d_name);
			printf("%s\n", file_path);
				
			//image im;
			//image sized = load_image_resize(input, net.w, net.h, net.c, &im);

			image im = load_image(file_path, 0, 0, net.c);
			image sized;
			if(letter_box) sized = letterbox_image(im, net.w, net.h);
			else sized = resize_image(im, net.w, net.h);

			layer l = net.layers[net.n - 1];
			int k;
			for (k = 0; k < net.n; ++k) {
				layer lk = net.layers[k];
				if (lk.type == YOLO || lk.type == GAUSSIAN_YOLO || lk.type == REGION) {
					l = lk;
					printf(" Detection layer: %d - type = %d \n", k, l.type);
				}
			}

			//box *boxes = calloc(l.w*l.h*l.n, sizeof(box));
			//float **probs = calloc(l.w*l.h*l.n, sizeof(float*));
			//for(j = 0; j < l.w*l.h*l.n; ++j) probs[j] = (float*)xcalloc(l.classes, sizeof(float));

			float *X = sized.data;

			//time= what_time_is_it_now();
			double time = get_time_point();
			network_predict(net, X);
			//network_predict_image(&net, im); letterbox = 1;
			// 원래 stdin으로 받아 input으로 넘긴 변수를 file_path 로 대신 사용합니다 
			printf("%s: Predicted in %lf milli-seconds.\n", file_path, ((double)get_time_point() - time) / 1000);
			//printf("%s: Predicted in %f seconds.\n", input, (what_time_is_it_now()-time));

			int nboxes = 0;
			detection *dets = get_network_boxes(&net, im.w, im.h, thresh, hier_thresh, 0, 1, &nboxes, letter_box);
			if (nms) {
				if (l.nms_kind == DEFAULT_NMS) do_nms_sort(dets, nboxes, l.classes, nms);
				else diounms_sort(dets, nboxes, l.classes, nms, l.nms_kind, l.beta_nms);
			}
			draw_detections_v3(im, dets, nboxes, thresh, names, alphabet, l.classes, ext_output);
			save_image(im, "predictions");
			if (!dont_show) {
				show_image(im, "predictions");
			}

			if (json_file) {
				if (json_buf) {
					char *tmp = ", \n";
					fwrite(tmp, sizeof(char), strlen(tmp), json_file);
				}
				++json_image_id;
				// 원래 stdin으로 받아 input으로 넘긴 변수를 file_path 로 대신 사용합니다 
				json_buf = detection_to_json(dets, nboxes, l.classes, names, json_image_id, file_path);

				fwrite(json_buf, sizeof(char), strlen(json_buf), json_file);
				free(json_buf);
			}

			// pseudo labeling concept - fast.ai
			if (save_labels)
			{
				char labelpath[4096];
				// 원래 stdin으로 받아 input으로 넘긴 변수를 file_path 로 대신 사용합니다, 추론결과를 YOLO 포맷의 라벨링 파일로 저장하는 구문입니다 
				replace_image_to_label(file_path, labelpath);

				FILE* fw = fopen(labelpath, "wb");
				int i;
				for (i = 0; i < nboxes; ++i) {
					char buff[1024];
					int class_id = -1;
					float prob = 0;
					for (j = 0; j < l.classes; ++j) {
						if (dets[i].prob[j] > thresh && dets[i].prob[j] > prob) {
							prob = dets[i].prob[j];
							class_id = j;
						}
					}
					if (class_id >= 0) {
						sprintf(buff, "%d %2.4f %2.4f %2.4f %2.4f\n", class_id, dets[i].bbox.x, dets[i].bbox.y, dets[i].bbox.w, dets[i].bbox.h);
						fwrite(buff, sizeof(char), strlen(buff), fw);
					}
				}
				fclose(fw);
			}

			free_detections(dets, nboxes);
			free_image(im);
			free_image(sized);

			if (!dont_show) {
				wait_until_press_key_cv();
				destroy_all_windows_cv();
			}
		}
		closedir (dir);
	} else {
	  /* could not open directory */
	  perror ("");
	  return EXIT_FAILURE;
	}
```

## 수정을 했으니 다시 빌드 합시다

In [ ]:
! cd darknet && make

# 빌드한 파일은 다음에 다시 빌드하지 않도록 저장합시다
- 런타임 연결이 해제되면 임시드라이브에 빌드한 파일은 사라집니다
- 여러분의 드라이브에 디렉토리 하나를 지정합시다

In [ ]:
import os
os.environ['MY_TRAIN_HOME'] = "/content/drive/MyDrive/mytrain"

In [ ]:
! mkdir -p $MY_TRAIN_HOME
! ln -s $MY_TRAIN_HOME /content/train
! cp /content/darknet/darknet $MY_TRAIN_HOME

# YOLOv4-tiny 학습에 필요한 초기 weights 를 다운로드 받습니다

In [ ]:
! mkdir -p /content/train/weights
! wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
! mv yolov4-tiny.conv.29 /content/train/weights

# 학습 모드로 구동해봅시다
- VOC2012 데이터세트는 PascalVOC 포맷으로 되어있기 때문에 YOLO포맷 변경이 필요하겠습니다
- YOLO 포맷에는 클래스 정의 파일이 별도로 필요합니다
- foot, hand, head 는 Main task에 없는 클래스 이므로 제외합니다
- VOC2012 Development Kit 문서를 확인해봐도 되고 bash커맨드로 확인할 수도 있습니다 

In [ ]:
! mkdir -p /content/train/cfg
! find /content/VOCdevkit/VOC2012/Annotations -name "*.xml" | xargs grep "<name>" | awk '{print $2}' | sed 's/<name>//g' | sed 's/<\/name>//g' | grep -v foot | grep -v hand | grep -v head | sort | uniq > /content/train/cfg/classes.txt
! cat /content/train/cfg/classes.txt

## VOC 2012 기본 데이터셋을 학습시켜봅시다
- 먼저 Pascal VOC 포맷을 YOLO 포맷으로 변경합니다
  - 오픈소스 갖다쓰기부터 시작합시다
- train/valid set을 나누어 리스트 파일을 생성합니다
- config 파일을 생성합니다

## Pascal VOC 포맷을 YOLO 포맷으로 변경
- 오픈소스 갖다쓰기
  - labelImg 오픈소스에서 Pascal VOC -> YOLO 변환 함수만 가져옵니다

-  전에 썼던 코드 재사용하기
  - Pascal VOC 읽기는 3주차에 많이 했으니 그 코드를 재사용 합시다

- 클래스 리스트 만들기
  - YOLO 에서는 클래스 리스트가 필수입니다
  - Annotation에서 클래스 이름이 아닌 클래스 인덱스가 기록되기 때문입니다

-  2개 쓰는 코드 만들기
  - 디렉토리를 순회하면서 Pascal VOC 라벨링 파일을 읽어서 YOLO 포맷으로 저장합시다

In [ ]:
# 이거는 다운로드 안되었을 때에만 다시 하기
# ! wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
# ! tar xvf VOCtrainval_11-May-2012.tar

In [ ]:
# labelImg 오픈소스에서 Pascal VOC -> YOLO 변환 함수
# self 대신 img_size를 파라미터로 넘기도록 변경합시다
def bnd_box_to_yolo_line(img_size, box, class_list=[]):
    x_min = box['xmin']
    x_max = box['xmax']
    y_min = box['ymin']
    y_max = box['ymax']

    x_center = float((x_min + x_max)) / 2 / img_size[1]
    y_center = float((y_min + y_max)) / 2 / img_size[0]

    w = float((x_max - x_min)) / img_size[1]
    h = float((y_max - y_min)) / img_size[0]

    # PR387
    box_name = box['name']
    if box_name not in class_list:
        class_list.append(box_name)

    class_index = class_list.index(box_name)

    return class_index, x_center, y_center, w, h

In [ ]:
# 3주차에 많이 했던 Pascal VOC 읽기
import cv2
import xml.etree.ElementTree as Et
from xml.etree.ElementTree import Element, ElementTree

def read_pascal_voc(filename):

    with open (filename, 'r', encoding='utf8') as xml:

        tree = Et.parse(xml)        
        root = tree.getroot()

        pascal_objects = []

        size = root.find("size")
        width = int(size.find("width").text)
        height = int(size.find("height").text)
        img_size = [height, width]

        objects = root.findall("object")
        for _object in objects:
            name = _object.find("name").text
            bndbox = _object.find("bndbox")

            xmin = int(float(bndbox.find("xmin").text))
            ymin = int(float(bndbox.find("ymin").text))
            xmax = int(float(bndbox.find("xmax").text))
            ymax = int(float(bndbox.find("ymax").text))

            box = {'xmin':xmin, 'ymin':ymin, 'xmax':xmax, 'ymax':ymax, 'name':name, 'img_size':img_size }
            pascal_objects.append(box)
        return img_size, pascal_objects

In [ ]:
import os
import json

# 원본 Pascal VOC annotation 경로
pascal_voc_annotation_dir = '/content/VOCdevkit/VOC2012/Annotations'

# darknet에서 읽을 YOLO annotation 경로
yolo_annotation_dir = '/content/VOCdevkit/VOC2012/labels'

# class 리스트 파일은 YOLO에서 필수입니다
class_list_file = '/content/train/cfg/classes.txt'

os.makedirs(yolo_annotation_dir, exist_ok=True)

class_list = []

with open (class_list_file, 'r', encoding='utf8') as classes_file:
    classes = classes_file.readlines()
    for c in classes:
        class_list.append(c.rstrip())

print(class_list)


for f in os.listdir(pascal_voc_annotation_dir):
    if f.endswith('.xml'):
        pascal_voc_file_path = os.path.join(pascal_voc_annotation_dir, f)
        print(pascal_voc_file_path)

        yolo_file_path = os.path.join(yolo_annotation_dir, os.path.basename(pascal_voc_file_path)[:-4] + '.txt')
        img_size, boxs = read_pascal_voc(pascal_voc_file_path)

        with open (yolo_file_path, 'w', encoding='utf8') as yolo_file:
            for box in boxs:      
                class_index, x_center, y_center, w, h = bnd_box_to_yolo_line(img_size, box=box, class_list=class_list)                
                yolo_file.write('{} {} {} {} {}\n'.format(class_index, x_center, y_center, w, h))
                print(yolo_file_path, class_index, x_center, y_center, w, h)
    else:
        print('{} is not Pascal VOC format'.format(f))

## train 데이터세트 리스트와 valid 데이터세트 리스트 만들기
- jpg 파일의 절대경로가 기록된 텍스트 파일입니다
- darknet에서 이 파일을 보고 이미지 경로를 받아옵니다
- 그리고 이 이미지가 있는 디렉토리를 기준으로 ../labels 디렉토리로부터 annotation 파일을 받아옵니다
  - darknet이 그렇게 동작하기 때문에 위에 소스코드에서도 YOLO annotation 변환경로를 설정했습니다
- train/valid 는 Main task에 정의된 것을 따릅니다

In [ ]:
# 이거는 다운로드 안되었을 때에만 다시 하기
# ! wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
# ! tar xvf VOCtrainval_11-May-2012.tar

! mkdir -p /content/train
! cat /content/VOCdevkit/VOC2012/ImageSets/Main/train.txt | awk '{printf "/content/VOCdevkit/VOC2012/JPEGImages/%s.jpg\n",$1}' > /content/train/cfg/train.list
! cat /content/VOCdevkit/VOC2012/ImageSets/Main/val.txt | awk '{printf "/content/VOCdevkit/VOC2012/JPEGImages/%s.jpg\n",$1}' > /content/train/cfg/valid.list

# 설정 파일을 만듭니다. 내용은 아래 셀의 내용이 들어갑니다
! echo "classes= 20" >> /content/train/cfg/train.data
! echo "train = /content/train/cfg/train.list" >> /content/train/cfg/train.data
! echo "valid  = /content/train/cfg/valid.list" >> /content/train/cfg/train.data
! echo "names = /content/train/cfg/classes.txt" >> /content/train/cfg/train.data
! echo "backup = /content/train/weights" >> /content/train/cfg/train.data

## 설정 파일 작성
- 학습 경로 설정파일(train.data)
  - 아래와 같이 저장해서 /content/train/train.data 또는 편하신 경로에 저장합니다
  - 단, 그 경로는 기억해둡니다

```
classes= 20
train = /content/train/cfg/train.list
valid  = /content/train/cfg/valid.list
names = /content/train/cfg/classes.txt

# weights 저장경로인데 google drive에 공간이 여유가 있다면 가급적 google drive 경로로 지정합니다
backup = /content/train/weights
```
- 모델 설정 파일(yolov4-tiny.cfg)
  - 원본 yolov4-tiny.cfg 에서 다음 사항을 수정합니다
  - [yolo] 라고 써진 layer로 스크롤을 내려봅니다
  - anchors
    - 아래 계산된 anchor로 수정
  - classes
    - 위에 작성한 classes.txt가 몇줄인지 확인하고 그 수로 수정합니다
  - [yolo] 라고 써진 layer 바로 위에 있는 [convolutional] layer의 filter(255라고 되어있을 것입니다) 를 다음 수로 수정
    - (num of classes + 5 ) x 3

## anchor 계산
- tiny-YOLOv4 기준으로 6개의 anchor가 필요하므로 train셋 기준으로 anchor를 재계산합니다
  - 여기서 계산된 anchor를 yolov4-tiny.cfg 에 적용합니다

In [ ]:
! cd /content/train && chmod 755 darknet && ./darknet detector calc_anchors /content/train/cfg/train.data -num_of_clusters 6 -width 416 -height 416

# yolov4-tiny 설정 파일을 수정합시다

In [ ]:
! cp /content/darknet/cfg/yolov4-tiny.cfg /content/train/cfg/

## 학습 시작


In [ ]:
! cd /content/train && ./darknet detector train /content/train/cfg/train.data /content/train/cfg/yolov4-tiny.cfg /content/train/weights/yolov4-tiny.conv.29 -dont_show

# mAP 평가
- 아래 용어를 혹시 모르신다면 참고해주세요
- IoU
  - Intersection Over Union
  - 여기서는 ground truth와 추론된 영역의 IoU를 의미합니다
- TP
  - True Positive
  - validation 데이터에 있는 오브젝트를 맞게 추론함
- FP
  - False Positive
  - validation 데이터에 없는 오브젝트를 추론함
- FN
  - False Negative
  - validation 데이터에 있는 오브젝트를 추론하지 못함
- 맞게 추론했다의 의미
  - mAP@0.50 => ground truth와 추론된 영역의 IoU가 0.5 이상인 경우 맞게 추론한 것으로 인정

In [ ]:
! cd /content/train/ &&  ./darknet detector map /content/train/cfg/train.data /content/train/cfg/yolov4-tiny.cfg /content/train/weights/yolov4-tiny_last.weights

## 결과가 어떤가요?
- 혹시 person을 제외한 나머지 클래스의 추론결과가 좋지 않은가요?
- 그럼 이제 여러분이 만든 데이터를 투입해봅시다